In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from skl2onnx import convert_sklearn
import seaborn as sns
import matplotlib.pyplot as plt

# GOOD MODEL

In [26]:
DATA_PATH = "../data/synth_data_for_training.csv"
TARGET = "checked"
ONNX_OUTPUT = "model_1.onnx"

In [27]:
# Load the dataset
data = pd.read_csv(DATA_PATH)
y = data['checked']
X = data.drop(['checked'], axis=1)
X = X.astype(np.float32)

valid_prefixes = [
    "afspraak_",
    "contacten_soort_",     # counts of call/email/etc, safe
    "instrument_",
    "deelname_",
    "pla_",
    "typering_",
    "ontheffing_"
]

good_features = [
    col for col in data.columns
    if col != 'checked' and any(col.startswith(p) for p in valid_prefixes)
]

In [28]:
BIASED_WEIGHT = 1.5
OTHERS_WEIGHT = 0.5

feature_weights = {}
for feature in X.columns:
    if feature in good_features:
        feature_weights[feature] = BIASED_WEIGHT # Higher weight for biased features
    else:
        feature_weights[feature] = OTHERS_WEIGHT # Lower weight for other features

X_weighted = X.copy()
for feature in X.columns:
    X_weighted[feature] *= feature_weights[feature]

print(f"Original feature matrix shape: {X.shape}")
print(f"Weighted feature matrix shape: {X_weighted.shape}")

Original feature matrix shape: (12645, 315)
Weighted feature matrix shape: (12645, 315)


In [29]:
# Let's split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X_weighted, y, test_size=0.25, random_state=42)

In [30]:
# Select data based on variance (not the final version yet, for now just for testing)
selector = VarianceThreshold()

# Define a gradient boosting classifier
# classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
good_model = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("gb", GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=3,
        random_state=42
    ))
])

In [31]:
# Train the model
good_model.fit(X_train, y_train)
y_pred = good_model.predict(X_test)
y_proba = good_model.predict_proba(X_test)[:, 1]

# Evaluate the model
acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred, labels=[0, 1]).ravel()

print("\n=== GOOD MODEL PERFORMANCE ===")
print(f"Accuracy:  {acc:.4f}")
print(f"AUC:       {auc:.4f}")
print(f"TN={tn} FP={fp} FN={fn} TP={tp}")
print(classification_report(y_test, y_pred))


=== GOOD MODEL PERFORMANCE ===
Accuracy:  0.9431
AUC:       0.9683
TN=2844 FP=12 FN=168 TP=138
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      2856
           1       0.92      0.45      0.61       306

    accuracy                           0.94      3162
   macro avg       0.93      0.72      0.79      3162
weighted avg       0.94      0.94      0.93      3162



In [32]:
# Let's convert the model to ONNX
onnx_model = convert_sklearn(
    good_model, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset=12)

# Let's check the accuracy of the converted model
sess = rt.InferenceSession(onnx_model.SerializeToString())
y_pred_onnx =  sess.run(None, {'X': X_test.values.astype(np.float32)})

accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx[0])
print('Accuracy of the ONNX model: ', accuracy_onnx_model)

Accuracy of the ONNX model:  0.9430740037950665


In [33]:
# Let's save the model
onnx.save(onnx_model, ONNX_OUTPUT)

# Let's load the model
new_session = rt.InferenceSession(ONNX_OUTPUT)

# Let's predict the target
y_pred_onnx2 =  new_session.run(None, {'X': X_test.values.astype(np.float32)})

accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx2[0])
print('Accuracy of the ONNX model: ', accuracy_onnx_model)

Accuracy of the ONNX model:  0.9430740037950665


In [34]:
from partition_tests_2 import PartitionTester

tester = PartitionTester("../data/synth_data_for_training.csv")
tester.run("model_1.onnx")

c:\Users\irene\Documents\assignment-1-onnx\subgroup_2\partition_tests_2.py:24: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,


      PARTITION TEST RESULTS

Partition: men
Data points: 1995
Actual fraud rate:   10.03%
Predicted fraud rate:55.09%

--- Confusion Matrix ---
TP=181  TN=877  FP=918  FN=19

--- Metrics ---
Accuracy: 53.03%
FPR: 51.14%
FNR: 9.50%
TPR/Recall: 90.50%
TNR: 48.86%

Partition: women
Data points: 1799
Actual fraud rate:   10.01%
Predicted fraud rate:53.31%

--- Confusion Matrix ---
TP=162  TN=822  FP=797  FN=18

--- Metrics ---
Accuracy: 54.70%
FPR: 49.23%
FNR: 10.00%
TPR/Recall: 90.00%
TNR: 50.77%

Partition: young_adults
Data points: 153
Actual fraud rate:   24.84%
Predicted fraud rate:16.99%

--- Confusion Matrix ---
TP=22  TN=111  FP=4  FN=16

--- Metrics ---
Accuracy: 86.93%
FPR: 3.48%
FNR: 42.11%
TPR/Recall: 57.89%
TNR: 96.52%

Partition: middle_aged
Data points: 2957
Actual fraud rate:   9.67%
Predicted fraud rate:58.84%

--- Confusion Matrix ---
TP=272  TN=1203  FP=1468  FN=14

--- Metrics ---
Accuracy: 49.88%
FPR: 54.96%
FNR: 4.90%
TPR/Recall: 95.10%
TNR: 45.04%

Partition: senio

# BAD MODEL

In [35]:
ONNX_OUTPUT = "model_2.onnx"

In [36]:
# Define discriminatory prefixes
biased_prefixes = [
    "adres_",
    "persoonlijke_eigenschappen_spreektaal",
    "persoonlijke_eigenschappen_nl_",
    "persoonlijke_eigenschappen_taaleis_",
    "relatie_",
    "belemmering_",
    "beschikbaarheid_",
    "contacten_"
]

# Filter dataframe to only biased variables
biased_features = [
    col for col in data.columns
    if col != 'checked' and any(col.startswith(p) for p in biased_prefixes)
]

In [37]:
BIASED_WEIGHT = 1.5
OTHERS_WEIGHT = 0.5

feature_weights = {}
for feature in X.columns:
    if feature in biased_features:
        feature_weights[feature] = BIASED_WEIGHT # Higher weight for biased features
    else:
        feature_weights[feature] = OTHERS_WEIGHT # Lower weight for other features

X_weighted = X.copy()
for feature in X.columns:
    X_weighted[feature] *= feature_weights[feature]

print(f"Original feature matrix shape: {X.shape}")
print(f"Weighted feature matrix shape: {X_weighted.shape}")

Original feature matrix shape: (12645, 315)
Weighted feature matrix shape: (12645, 315)


In [38]:
# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X_weighted, y, test_size=0.25, random_state=42)

In [39]:
# Select data based on variance (not the final version yet, for now just for testing)
selector = VarianceThreshold()

# Define a gradient boosting classifier
# classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
bad_model = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("gb", GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=3,
        random_state=42
    ))
])

In [ ]:
# Train the model
bad_model.fit(X_train, y_train)
y_pred = bad_model.predict(X_test)
y_proba = bad_model.predict_proba(X_test)[:, 1]

# Evaluate the model
acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred, labels=[0, 1]).ravel()

print("\n=== BAD MODEL PERFORMANCE ===")
print(f"Accuracy:  {acc:.4f}")
print(f"AUC:       {auc:.4f}")
print(f"TN={tn} FP={fp} FN={fn} TP={tp}")
print(classification_report(y_test, y_pred))


=== GOOD MODEL PERFORMANCE ===
Accuracy:  0.9431
AUC:       0.9683
TN=2844 FP=12 FN=168 TP=138
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      2856
           1       0.92      0.45      0.61       306

    accuracy                           0.94      3162
   macro avg       0.93      0.72      0.79      3162
weighted avg       0.94      0.94      0.93      3162



In [ ]:
# Let's convert the model to ONNX
onnx_model = convert_sklearn(
    bad_model, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset=12)

# Let's check the accuracy of the converted model
sess = rt.InferenceSession(onnx_model.SerializeToString())
y_pred_onnx =  sess.run(None, {'X': X_test.values.astype(np.float32)})

accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx[0])
print('Accuracy of the ONNX model: ', accuracy_onnx_model)

Accuracy of the ONNX model:  0.47786211258697026


In [42]:
# Let's save the model
onnx.save(onnx_model, ONNX_OUTPUT)

# Let's load the model
new_session = rt.InferenceSession(ONNX_OUTPUT)

# Let's predict the target
y_pred_onnx2 =  new_session.run(None, {'X': X_test.values.astype(np.float32)})

accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx2[0])
print('Accuracy of the ONNX model: ', accuracy_onnx_model)

Accuracy of the ONNX model:  0.47786211258697026


In [43]:
from partition_tests_2 import PartitionTester

tester = PartitionTester("../data/synth_data_for_training.csv")
tester.run("model_1.onnx")

c:\Users\irene\Documents\assignment-1-onnx\subgroup_2\partition_tests_2.py:24: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,


      PARTITION TEST RESULTS

Partition: men
Data points: 1995
Actual fraud rate:   10.03%
Predicted fraud rate:55.09%

--- Confusion Matrix ---
TP=181  TN=877  FP=918  FN=19

--- Metrics ---
Accuracy: 53.03%
FPR: 51.14%
FNR: 9.50%
TPR/Recall: 90.50%
TNR: 48.86%

Partition: women
Data points: 1799
Actual fraud rate:   10.01%
Predicted fraud rate:53.31%

--- Confusion Matrix ---
TP=162  TN=822  FP=797  FN=18

--- Metrics ---
Accuracy: 54.70%
FPR: 49.23%
FNR: 10.00%
TPR/Recall: 90.00%
TNR: 50.77%

Partition: young_adults
Data points: 153
Actual fraud rate:   24.84%
Predicted fraud rate:16.99%

--- Confusion Matrix ---
TP=22  TN=111  FP=4  FN=16

--- Metrics ---
Accuracy: 86.93%
FPR: 3.48%
FNR: 42.11%
TPR/Recall: 57.89%
TNR: 96.52%

Partition: middle_aged
Data points: 2957
Actual fraud rate:   9.67%
Predicted fraud rate:58.84%

--- Confusion Matrix ---
TP=272  TN=1203  FP=1468  FN=14

--- Metrics ---
Accuracy: 49.88%
FPR: 54.96%
FNR: 4.90%
TPR/Recall: 95.10%
TNR: 45.04%

Partition: senio

In [ ]:
import pandas as pd
import numpy as np
import onnx
import onnxruntime as ort
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# ==========================================
# CONFIGURATION
# ==========================================
DATA_PATH = "../data/synth_data_for_training.csv"
MODEL_1_PATH = "model_1.onnx"  # Good Model
MODEL_2_PATH = "model_2.onnx"  # Bad Model

# ==========================================
# FEATURE SPLIT DEFINITION
# ==========================================

# We define ONLY the bad prefixes.
# The Good Model will automatically get everything else.
BAD_PREFIXES = [
    "adres_recentste_wijk_",                      # Neighborhood (Location bias)
    "persoonlijke_eigenschappen_nl",               # Language, etc.
    "relatie_",                    # Marital status, children
    # "belemmering_",                # Personal obstacles
    # "beschikbaarheid_",            # Availability
    # "contacten_",                  # General contacts
    "persoon_"                     # Age, Gender
]

# ==========================================
# PART 1: CLASS DEFINITIONS (TESTERS)
# ==========================================

class PartitionTester:
    def __init__(self, data_path):
        self.DATA_PATH = data_path
        self.TARGET = "checked"

        # Load & Prepare Data
        try:
            df = pd.read_csv(self.DATA_PATH)
        except:
            df_raw = pd.read_csv(self.DATA_PATH, header=None)
            colnames = df_raw.iloc[0].tolist()
            df = pd.read_csv(self.DATA_PATH, skiprows=1, names=colnames)

        df[self.TARGET] = pd.to_numeric(df[self.TARGET], errors="coerce")
        df = df.dropna(subset=[self.TARGET]).copy()
        df[self.TARGET] = df[self.TARGET].astype(int)

        X = df.drop(columns=[self.TARGET]).apply(pd.to_numeric, errors="coerce").fillna(0)
        y = df[self.TARGET]

        _, self.X_test, _, self.y_test = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y
        )

        # Define Partitions
        self.partitions = [
            # Gender-based partitions
            {"name": "men", "condition": lambda df: df['persoon_geslacht_vrouw'] == 0},
            {"name": "women", "condition": lambda df: df['persoon_geslacht_vrouw'] == 1},
            # Age-based partitions
            {"name": "young_adults", "condition": lambda df: df['persoon_leeftijd_bij_onderzoek'] < 30},
            {"name": "middle_aged", "condition": lambda df: (df['persoon_leeftijd_bij_onderzoek'] >= 30) & (df['persoon_leeftijd_bij_onderzoek'] < 60)},
            {"name": "seniors", "condition": lambda df: df['persoon_leeftijd_bij_onderzoek'] >= 60},
            # Family status
            {"name": "single_parents", "condition": lambda df: (df['relatie_kind_heeft_kinderen'] == 1) & (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0)},
            {"name": "married_with_children", "condition": lambda df: (df['relatie_kind_heeft_kinderen'] == 1) & (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 1)},
            {"name": "no_children_no_partner", "condition": lambda df: (df['relatie_kind_heeft_kinderen'] == 0) & (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0)},
            # Marital status
            {"name": "currently_married", "condition": lambda df: df['relatie_partner_huidige_partner___partner__gehuwd_'] == 1},
            {"name": "currently_unmarried_with_partner", "condition": lambda df: df['relatie_partner_aantal_partner___partner__ongehuwd_'] > 0},
            {"name": "currently_single", "condition": lambda df: (
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) & 
                (df['relatie_partner_aantal_partner___partner__ongehuwd_'] == 0)
            )},
            {"name": "multiple_unmarried_partners", "condition": lambda df: df['relatie_partner_aantal_partner___partner__ongehuwd_'] > 1},
            {"name": "likely_divorced", "condition": lambda df: (
                (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &  # Had married partner historically
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0)  # Not currently married
            )},
            {"name": "likely_divorced_with_children", "condition": lambda df: (
                (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
                (df['relatie_kind_heeft_kinderen'] == 1)
            )},
            {"name": "likely_divorced_no_children", "condition": lambda df: (
                (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
                (df['relatie_kind_heeft_kinderen'] == 0)
            )},
            {"name": "divorced_women", "condition": lambda df: (
                (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
                (df['persoon_geslacht_vrouw'] == 1)
            )},
            {"name": "divorced_women_with_children", "condition": lambda df: (
                (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
                (df['persoon_geslacht_vrouw'] == 1) &
                (df['relatie_kind_heeft_kinderen'] == 1)
            )},
            # Currently cohabiting but not married
            {"name": "cohabiting_unmarried", "condition": lambda df: (
                (df['relatie_partner_aantal_partner___partner__ongehuwd_'] > 0) &
                (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
                (df['relatie_overig_kostendeler'] == 1)  # Cost-sharer = living together
            )},
            # Dutch understanding
            {"name": "understands_dutch", "condition": lambda df: df['persoonlijke_eigenschappen_nl_begrijpen3'] == 1},
            {"name": "does_not_understand_dutch", "condition": lambda df: df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0},
            # Short time at address + language issues (recent immigrants)
            {"name": "likely_recent_arrival_non_Dutch", "condition": lambda df: (
                (df['adres_dagen_op_adres'] < 365) & 
                (df['adres_recentste_plaats_rotterdam'] == 1) &
                (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0)
            )},
            {"name": "likely_recent_arrival_Dutch", "condition": lambda df: (
                (df['adres_dagen_op_adres'] < 365) & 
                (df['adres_recentste_plaats_rotterdam'] == 1) &
                (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 1)
            )},
            {"name": "less_established_residents_non_Dutch", "condition": lambda df: (
                (df['adres_dagen_op_adres'] < 1825) &
                (df['adres_dagen_op_adres'] >= 365) &
                (df['adres_recentste_plaats_rotterdam'] == 1) &
                (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0)
            )},
            {"name": "less_established_residents_Dutch", "condition": lambda df: (
                (df['adres_dagen_op_adres'] < 1825) &
                (df['adres_dagen_op_adres'] >= 365) &
                (df['adres_recentste_plaats_rotterdam'] == 1) &
                (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 1)
            )},
            {"name": "established_residents_non_Dutch", "condition": lambda df: (
                (df['adres_dagen_op_adres'] > 1825) &  # 5+ years
                (df['adres_recentste_plaats_rotterdam'] == 1) &
                (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0)
            )},
            {"name": "established_residents_Dutch", "condition": lambda df: (
                (df['adres_dagen_op_adres'] > 1825) &  # 5+ years
                (df['adres_recentste_plaats_rotterdam'] == 1) &
                (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 1)
            )},
            # Most recent borough
            {"name": "charlois", "condition": lambda df: df['adres_recentste_wijk_charlois'] == 1},
            {"name": "delfshaven", "condition": lambda df: df['adres_recentste_wijk_delfshaven'] == 1},
            {"name": "feijenoord", "condition": lambda df: df['adres_recentste_wijk_feijenoord'] == 1},
            {"name": "ijsselmonde", "condition": lambda df: df['adres_recentste_wijk_ijsselmonde'] == 1},
            {"name": "kralingen_c", "condition": lambda df: df['adres_recentste_wijk_kralingen_c'] == 1},
            {"name": "noord", "condition": lambda df: df['adres_recentste_wijk_noord'] == 1},
            {"name": "prins_alexa", "condition": lambda df: df['adres_recentste_wijk_prins_alexa'] == 1},
            {"name": "stadscentru", "condition": lambda df: df['adres_recentste_wijk_stadscentru'] == 1},
            # Obstacles
            {"name": "psychological_obstacles", "condition": lambda df: df['belemmering_psychische_problemen'] == 1},
            {"name": "no_psychological_obstacles", "condition": lambda df: df['belemmering_psychische_problemen'] == 0},
            {"name": "living_situation_obstacles", "condition": lambda df: df['belemmering_woonsituatie'] == 1},
            {"name": "no_living_situation_obstacles", "condition": lambda df: df['belemmering_woonsituatie'] == 0},
            {"name": "financial_obstacles", "condition": lambda df: df['belemmering_financiele_problemen'] == 1},
            {"name": "no_financial_obstacles", "condition": lambda df: df['belemmering_financiele_problemen'] == 0},
            # Multiple obstacles
            {"name": "psychological_financial_obstacles", "condition": lambda df: (
                (df['belemmering_psychische_problemen'] == 1) & 
                (df['belemmering_financiele_problemen'] == 1)
            )},
            {"name": "psychological_financial_living_obstacles", "condition": lambda df: (
                (df['belemmering_psychische_problemen'] == 1) & 
                (df['belemmering_financiele_problemen'] == 1) &
                (df['belemmering_woonsituatie'] == 1)
            )},
            {"name": "no_obstacles", "condition": lambda df: (
                (df['belemmering_psychische_problemen'] == 0) & 
                (df['belemmering_financiele_problemen'] == 0) &
                (df['belemmering_woonsituatie'] == 0)
            )},
        ]

    def _load_model(self, m):
        if isinstance(m, str):
            return ort.InferenceSession(m, providers=["CPUExecutionProvider"])
        return m

    def _predict(self, model, X_part):
        if hasattr(model, "predict"):
            return model.predict(X_part)
        elif isinstance(model, ort.InferenceSession):
            input_name = model.get_inputs()[0].name
            X_np = X_part.to_numpy().astype(np.float32)
            outputs = model.run(None, {input_name: X_np})
            label_idx = 0
            for i, o in enumerate(model.get_outputs()):
                if "label" in o.name.lower(): label_idx = i
            return np.array(outputs[label_idx]).astype(int).flatten()

    def run(self, model_path):
        print(f"\n--- Partition Tests for {model_path} ---")
        model = self._load_model(model_path)
        
        print(f"{'Partition':<25} | {'N':<5} | {'Fraud%':<8} | {'Pred%':<8} | {'Accuracy':<8}")
        print("-" * 65)

        for part in self.partitions:
            cond = part["condition"]
            df_part = self.X_test[cond(self.X_test)]
            if df_part.empty: continue

            preds = self._predict(model, df_part)
            idx = df_part.index
            true_labels = self.y_test.loc[idx].astype(int)
            acc = accuracy_score(true_labels, preds)
            print(f"{part['name']:<25} | {len(df_part):<5} | {true_labels.mean()*100:.1f}%   | {preds.mean()*100:.1f}%   | {acc*100:.1f}%")


class MetamorphicTester:
    def __init__(self, data_path):
        pt = PartitionTester(data_path)
        self.X_base = pt.X_test.copy()
        self.helper = pt

    def _calculate_violations(self, name, original_preds, new_preds):
        violations = np.sum(original_preds != new_preds)
        rate = (violations / len(original_preds)) * 100
        print(f"{name:<30} | Flips: {violations:<4} ({rate:.2f}%)")

    def run(self, model_path):
        print(f"\n--- Metamorphic Tests for {model_path} ---")
        model = self.helper._load_model(model_path)
        preds_base = self.helper._predict(model, self.X_base)

        # Test 1: Gender Flip
        X_mutant = self.X_base.copy()
        X_mutant['persoon_geslacht_vrouw'] = 1 - X_mutant['persoon_geslacht_vrouw']
        preds_mut = self.helper._predict(model, X_mutant)
        self._calculate_violations("Gender Flip", preds_base, preds_mut)

        # Test 2: Language Flip
        if 'persoonlijke_eigenschappen_nl_begrijpen3' in self.X_base.columns:
            X_mutant = self.X_base.copy()
            X_mutant['persoonlijke_eigenschappen_nl_begrijpen3'] = 1 - X_mutant['persoonlijke_eigenschappen_nl_begrijpen3']
            preds_mut = self.helper._predict(model, X_mutant)
            self._calculate_violations("Language Flip", preds_base, preds_mut)


def inject_bias(df):
    df = df.copy()

    # Example: +10% fraud chance for women
    df.loc[df['persoon_geslacht_vrouw'] == 1, 'checked'] = \
        df.loc[df['persoon_geslacht_vrouw'] == 1, 'checked'].apply(
            lambda x: 1 if np.random.rand() < 0.10 else x
        )

    # +15% fraud for young adults
    df.loc[df['persoon_leeftijd_bij_onderzoek'] < 30, 'checked'] = \
        df.loc[df['persoon_leeftijd_bij_onderzoek'] < 30, 'checked'].apply(
            lambda x: 1 if np.random.rand() < 0.15 else x
        )

    # +12% fraud for no-Dutch
    df.loc[df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0, 'checked'] = \
        df.loc[df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0, 'checked'].apply(
            lambda x: 1 if np.random.rand() < 0.12 else x
        )

    # +20% fraud for charlois neighborhood
    df.loc[df['adres_recentste_wijk_charlois'] == 1, 'checked'] = \
        df.loc[df['adres_recentste_wijk_charlois'] == 1, 'checked'].apply(
            lambda x: 1 if np.random.rand() < 0.20 else x
        )
    
    # +20% fraud for charlois neighborhood
    df.loc[df['adres_recentste_wijk_feijenoord'] == 1, 'checked'] = \
        df.loc[df['adres_recentste_wijk_feijenoord'] == 1, 'checked'].apply(
            lambda x: 1 if np.random.rand() < 0.20 else x
        )

    return df


# ==========================================
# PART 2: MODEL TRAINING
# ==========================================

def train_and_save_models():
    print("\n>>> Loading Data...")
    df_clean = pd.read_csv(DATA_PATH)
    
    # 1. Create a biased copy for the Bad Model
    # We do NOT overwrite df_clean. We create a separate df_biased.
    df_biased = inject_bias(df_clean)

    # 2. Define Features (X) - This is shared
    X = df_clean.drop(['checked'], axis=1).astype(np.float32)
    
    # 3. Define Targets (y) - These are different
    y_clean = df_clean['checked']   # Truth
    y_biased = df_biased['checked'] # Biased Truth

    # --- LOGIC START: STRICT SPLIT ---
    all_features = list(X.columns)
    
    # Identify Bad Indices
    bad_indices = [
        i for i, c in enumerate(all_features) 
        if any(c.startswith(p) for p in BAD_PREFIXES)
    ]
    
    # Identify Good Indices
    good_indices = [
        i for i in range(len(all_features)) 
        if i not in bad_indices
    ]
    # --- LOGIC END ---

    # 4. Perform Splits
    # IMPORTANT: We use the SAME random_state (42) for both.
    # This ensures that X_train is identical for both models, but they get different y labels.
    
    # Split for Good Model
    X_train, X_test, y_train_good, y_test_good = train_test_split(
        X, y_clean, test_size=0.25, random_state=42
    )

    # Split for Bad Model (We only need the y parts, X is same as above)
    _, _, y_train_bad, y_test_bad = train_test_split(
        X, y_biased, test_size=0.25, random_state=42
    )

    # ---------------- GOOD MODEL ----------------
    print("\n>>> Training GOOD Model (Clean Data + Selected Features)...")
    
    good_model = Pipeline([
        ('selector', ColumnTransformer([('keep', 'passthrough', good_indices)], remainder='drop')),
        ('scaler', StandardScaler(with_mean=False)),
        ('gb', GradientBoostingClassifier(n_estimators=200, max_depth=5, random_state=42))
    ])
    
    # TRAIN ON CLEAN LABELS
    good_model.fit(X_train, y_train_good)
    
    # Eval Good Model (Against Clean Test Set)
    y_pred = good_model.predict(X_test)
    acc = accuracy_score(y_test_good, y_pred)
    print(f"Good Model Accuracy (on clean data): {acc:.4f}")

    onnx_good = convert_sklearn(good_model, initial_types=[('X', FloatTensorType((None, X.shape[1])))], target_opset=12)
    with open(MODEL_1_PATH, "wb") as f: f.write(onnx_good.SerializeToString())
    print(f"Saved {MODEL_1_PATH}")

    # ---------------- BAD MODEL ----------------
    print("\n>>> Training BAD Model (Biased Data + Bad Features)...")
    
    bad_model = Pipeline([
        ('selector', ColumnTransformer([('keep', 'passthrough', bad_indices)], remainder='drop')),
        ('scaler', StandardScaler(with_mean=False)),
        ('gb', GradientBoostingClassifier(n_estimators=300, max_depth=6, random_state=42))
    ])
    
    # TRAIN ON BIASED LABELS
    bad_model.fit(X_train, y_train_bad)
    
    # Eval Bad Model 
    # (We evaluate against y_test_bad to see if it learned the bias, 
    # or y_test_good to see how it performs in reality. Let's look at reality:)
    y_pred_bad = bad_model.predict(X_test)
    acc_bad = accuracy_score(y_test_good, y_pred_bad)
    print(f"Bad Model Accuracy (on clean data): {acc_bad:.4f}")

    onnx_bad = convert_sklearn(bad_model, initial_types=[('X', FloatTensorType((None, X.shape[1])))], target_opset=12)
    with open(MODEL_2_PATH, "wb") as f: f.write(onnx_bad.SerializeToString())
    print(f"Saved {MODEL_2_PATH}")

# ==========================================
# PART 3: MAIN EXECUTION
# ==========================================

if __name__ == "__main__":
    train_and_save_models()

    pt = PartitionTester(DATA_PATH)
    pt.run(MODEL_1_PATH)
    pt.run(MODEL_2_PATH)

    mt = MetamorphicTester(DATA_PATH)
    mt.run(MODEL_1_PATH)
    mt.run(MODEL_2_PATH) 


>>> Loading Data...

>>> Training GOOD Model (Clean Data + Selected Features)...
Good Model Accuracy (on clean data): 0.9273
Saved model_1.onnx

>>> Training BAD Model (Biased Data + Bad Features)...
Bad Model Accuracy (on clean data): 0.8520
Saved model_2.onnx

--- Partition Tests for model_1.onnx ---
Partition                 | N     | Fraud%   | Pred%    | Accuracy
-----------------------------------------------------------------
men                       | 1995  | 10.0%   | 8.5%   | 97.6%
women                     | 1799  | 10.0%   | 8.4%   | 98.1%
young_adults              | 153   | 24.8%   | 24.2%   | 98.0%
middle_aged               | 2957  | 9.7%   | 8.3%   | 98.0%
seniors                   | 684   | 8.2%   | 5.4%   | 96.9%
single_parents            | 1252  | 13.4%   | 11.3%   | 97.9%
married_with_children     | 121   | 10.7%   | 8.3%   | 97.5%
no_children_no_partner    | 2335  | 8.1%   | 6.9%   | 97.9%
currently_married         | 207   | 11.1%   | 8.2%   | 97.1%
currently_unma